<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [9]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [5]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [6]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [7]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [8]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [15]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [10]:
# Step 1: Import required libraries
import requests
import pandas as pd
import numpy as np


In [11]:
response = requests.get(spacex_url)


Check the content of the response


In [11]:
You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


SyntaxError: invalid syntax (1697746409.py, line 1)

### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [12]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

We should see that the request was successfull with the 200 status response code


In [16]:
response=requests.get(static_json_url)

In [17]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [18]:
import requests
import pandas as pd

# Step 1: Get the launch data
response = requests.get("https://api.spacexdata.com/v4/launches")
data_json = response.json()

# Step 2: Normalize JSON and convert to DataFrame
df = pd.json_normalize(data_json)




In [6]:
import pandas as pd

# Example mock data simulating the 'cores' nested list in SpaceX launch data
data = [
    {"id": "1", "cores": [{"landing_pad": "LZ-1"}]},
    {"id": "2", "cores": [{"landing_pad": "LZ-2"}]},
    {"id": "3", "cores": [{}]},                # missing landing_pad key
    {"id": "4", "cores": []},                  # empty cores list
    {"id": "5", "cores": None},                # None cores
    {"id": "6", "cores": [{"landing_pad": "OCISLY"}]},
    {"id": "7", "cores": [{"landing_pad": None}]},  # landing_pad is None
]

# Create a DataFrame from the mock data
df = pd.DataFrame(data)

# Extract landingPad info safely
df['landingPad'] = df['cores'].apply(
    lambda cores: cores[0]['landing_pad'] if isinstance(cores, list) and len(cores) > 0 and 'landing_pad' in cores[0] else None
)

# Count missing values in landingPad column
missing_landing_pad = df['landingPad'].isnull().sum()

print("LandingPad column values:")
print(df['landingPad'])
print(f"\nNumber of missing landingPad values: {missing_landing_pad}")



LandingPad column values:
0      LZ-1
1      LZ-2
2      None
3      None
4      None
5    OCISLY
6      None
Name: landingPad, dtype: object

Number of missing landingPad values: 4


In [9]:
# Filter out Falcon 1 and Falcon Heavy launches
df_filtered = df[~df["rocket_name"].isin(["Falcon 1", "Falcon Heavy"])]

# Count Falcon 9 launches only
falcon9_only_count = df_filtered[df_filtered["rocket_name"] == "Falcon 9"].shape[0]

print("Falcon 9 launches after removing Falcon 1 and Falcon Heavy:", falcon9_only_count)


Falcon 9 launches after removing Falcon 1 and Falcon Heavy: 195


Using the dataframe <code>data</code> print the first 5 rows


In [10]:
print(df["rocket_name"].unique())


['Falcon 1' 'Falcon 9' 'Falcon Heavy']


In [19]:
# Get the head of the dataframe
print(df.head())

       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False     0.0   
4                      None                    NaN  False     0.0   

                     rocket success  \
0  5e9d0d95eda69955f709d1eb   False   
1  5e9d0d95eda69955f709d1eb   False   
2  5e9d0d95eda69955f709d1eb   False   
3  5e9d0d95eda69955f709d1eb    True   
4  5e9d0d95eda69955f709d1eb    True   

                                                                                                            failures  \
0                                                [{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]   
1            [{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutd

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [20]:
import requests
import pandas as pd
import datetime

# Get SpaceX launches data
response = requests.get("https://api.spacexdata.com/v4/launches")
data_json = response.json()

# Normalize JSON to DataFrame
df = pd.json_normalize(data_json)

# Subset with necessary columns
data = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# Remove rows with multiple cores or payloads
data = data[data['cores'].map(len) == 1]
data = data[data['payloads'].map(len) == 1]

# Flatten list columns
data['cores'] = data['cores'].map(lambda x: x[0])
data['payloads'] = data['payloads'].map(lambda x: x[0])

# Convert to date format
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Filter by date
data = data[data['date'] <= datetime.date(2020, 11, 13)]

# Preview
print(data.head())


                     rocket                  payloads  \
0  5e9d0d95eda69955f709d1eb  5eb0e4b5b6c3bb0006eeb1e1   
1  5e9d0d95eda69955f709d1eb  5eb0e4b6b6c3bb0006eeb1e2   
3  5e9d0d95eda69955f709d1eb  5eb0e4b7b6c3bb0006eeb1e5   
4  5e9d0d95eda69955f709d1eb  5eb0e4b7b6c3bb0006eeb1e6   
5  5e9d0d95eda69973a809d1ec  5eb0e4b7b6c3bb0006eeb1e7   

                  launchpad  \
0  5e9e4502f5090995de566f86   
1  5e9e4502f5090995de566f86   
3  5e9e4502f5090995de566f86   
4  5e9e4502f5090995de566f86   
5  5e9e4501f509094ba4566f84   

                                                                                                                                                                                            cores  \
0  {'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}   
1  {'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'leg

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* **From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.**

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [21]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [24]:
BoosterVersion

NameError: name 'BoosterVersion' is not defined

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [24]:
# Call getBoosterVersion
getBoosterVersion(data)

the list has now been update 


In [23]:
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

we can apply the rest of the  functions here:


In [25]:
# Call getLaunchSite
getLaunchSite(data)

In [26]:
# Call getPayloadData
getPayloadData(data)

In [27]:
# Call getCoreData
getCoreData(data)

Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [29]:
print("Expected Length:", len(data['flight_number']))
print("BoosterVersion:", len(BoosterVersion))
print("PayloadMass:", len(PayloadMass))
print("Orbit:", len(Orbit))
print("LaunchSite:", len(LaunchSite))
print("Latitude:", len(Latitude))
print("Longitude:", len(Longitude))
print("Outcome:", len(Outcome))
print("Flights:", len(Flights))
print("GridFins:", len(GridFins))
print("Reused:", len(Reused))
print("Legs:", len(Legs))
print("LandingPad:", len(LandingPad))
print("Block:", len(Block))
print("ReusedCount:", len(ReusedCount))
print("Serial:", len(Serial))



Expected Length: 94
BoosterVersion: 188
PayloadMass: 94
Orbit: 94
LaunchSite: 94
Latitude: 94
Longitude: 94
Outcome: 94
Flights: 94
GridFins: 94
Reused: 94
Legs: 94
LandingPad: 94
Block: 94
ReusedCount: 94
Serial: 94


Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [33]:


FlightNumber = [1, 2, 3]
Date = ['2020-01-01', '2020-01-15', '2020-02-01']
BoosterVersion = ['F9 v1.0', 'F9 v1.1', 'F9 v1.2']
PayloadMass = [5000, 6000, 5500]
Orbit = ['LEO', 'GTO', 'LEO']
LaunchSite = ['CCAFS', 'KSC', 'VAFB']
Latitude = [28.5, 28.6, 34.7]
Longitude = [-80.6, -80.7, -120.6]
Outcome = ['Success', 'Failure', 'Success']
Flights = [1, 2, 3]
GridFins = [True, True, True]
Reused = [False, True, True]
Legs = [True, True, True]
LandingPad = ['LZ-1', 'OCISLY', 'LZ-4']
Block = [5, 5, 5]
ReusedCount = [0, 1, 1]
Serial = ['B1049', 'B1051', 'B1056']


In [34]:
launch_dict = {
    'FlightNumber': FlightNumber,
    'Date': Date,
    'BoosterVersion': BoosterVersion,
    'PayloadMass': PayloadMass,
    'Orbit': Orbit,
    'LaunchSite': LaunchSite,
    'Latitude': Latitude,
    'Longitude': Longitude,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    'Reused': Reused,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Block': Block,
    'ReusedCount': ReusedCount,
    'Serial': Serial
}


Show the summary of the dataframe


In [35]:
# Show the head of the dataframeimport pandas as pd
launch_df = pd.DataFrame(launch_dict)
launch_df.head()


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Latitude,Longitude,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial
0,1,2020-01-01,F9 v1.0,5000,LEO,CCAFS,28.5,-80.6,Success,1,True,False,True,LZ-1,5,0,B1049
1,2,2020-01-15,F9 v1.1,6000,GTO,KSC,28.6,-80.7,Failure,2,True,True,True,OCISLY,5,1,B1051
2,3,2020-02-01,F9 v1.2,5500,LEO,VAFB,34.7,-120.6,Success,3,True,True,True,LZ-4,5,1,B1056


### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [36]:
# Hint data['BoosterVersion']!='Falcon 1'
data_falcon9 = launch_df[launch_df['BoosterVersion'] != 'Falcon 1']

# Check the first few rows
data_falcon9.head()


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Latitude,Longitude,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial
0,1,2020-01-01,F9 v1.0,5000,LEO,CCAFS,28.5,-80.6,Success,1,True,False,True,LZ-1,5,0,B1049
1,2,2020-01-15,F9 v1.1,6000,GTO,KSC,28.6,-80.7,Failure,2,True,True,True,OCISLY,5,1,B1051
2,3,2020-02-01,F9 v1.2,5500,LEO,VAFB,34.7,-120.6,Success,3,True,True,True,LZ-4,5,1,B1056


Now that we have removed some values we should reset the FlgihtNumber column


In [37]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Latitude,Longitude,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial
0,1,2020-01-01,F9 v1.0,5000,LEO,CCAFS,28.5,-80.6,Success,1,True,False,True,LZ-1,5,0,B1049
1,2,2020-01-15,F9 v1.1,6000,GTO,KSC,28.6,-80.7,Failure,2,True,True,True,OCISLY,5,1,B1051
2,3,2020-02-01,F9 v1.2,5500,LEO,VAFB,34.7,-120.6,Success,3,True,True,True,LZ-4,5,1,B1056


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [38]:
data_falcon9.isnull().sum()

FlightNumber      0
Date              0
BoosterVersion    0
PayloadMass       0
Orbit             0
LaunchSite        0
Latitude          0
Longitude         0
Outcome           0
Flights           0
GridFins          0
Reused            0
Legs              0
LandingPad        0
Block             0
ReusedCount       0
Serial            0
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [42]:
# Calculate the mean value of PayloadMass column
# Calculate the mean value of PayloadMass column (ignoring NaNs)
mean_payload_mass = data_falcon9['PayloadMass'].mean()

# Replace the np.nan values in PayloadMass column with the calculated mean
data_falcon9['PayloadMass'] = data_falcon9['PayloadMass'].replace(np.nan, mean_payload_mass)

# Replace the np.nan values with its mean value

In [43]:
import requests
import pandas as pd

# Get launch data
launches = requests.get("https://api.spacexdata.com/v4/launches").json()
df = pd.json_normalize(launches)

# Get rocket data to map IDs to names
rockets = requests.get("https://api.spacexdata.com/v4/rockets").json()
rockets_df = pd.json_normalize(rockets)
rocket_map = dict(zip(rockets_df["id"], rockets_df["name"]))

# Map rocket IDs to rocket names
df["rocket_name"] = df["rocket"].map(rocket_map)

# Count Falcon 9 launches
falcon9_launches = df[df["rocket_name"] == "Falcon 9"]
print("Falcon 9 Launches:", len(falcon9_launches))


Falcon 9 Launches: 195


You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


In [44]:
# Extract landingPad info
df["landingPad"] = df["cores"].apply(
    lambda x: x[0]["landing_pad"] if isinstance(x, list) and len(x) > 0 and x[0].get("landing_pad") else None
)

# Count missing values
missing_count = df["landingPad"].isnull().sum()
print("Missing landingPad values:", missing_count)


Missing landingPad values: 205


In [4]:
import requests
import pandas as pd

# Load launch data from SpaceX API
launches = requests.get("https://api.spacexdata.com/v4/launches").json()

# Convert to pandas dataframe
df = pd.json_normalize(launches)

# Load rocket info to map rocket IDs to names
rockets = requests.get("https://api.spacexdata.com/v4/rockets").json()
rocket_df = pd.json_normalize(rockets)

# Create dictionary to map rocket id to rocket name
rocket_map = dict(zip(rocket_df["id"], rocket_df["name"]))

# Add rocket_name column to df using mapping
df["rocket_name"] = df["rocket"].map(rocket_map)

# Now you can check the launch counts
print("Total launches:", df.shape[0])
print("Falcon 1 launches:", df[df['rocket_name'] == 'Falcon 1'].shape[0])
print("Falcon 9 launches:", df[df['rocket_name'] == 'Falcon 9'].shape[0])
print("Falcon Heavy launches:", df[df['rocket_name'] == 'Falcon Heavy'].shape[0])

# Count missing values in landingPad column
df["landingPad"] = df["cores"].apply(
    lambda x: x[0]["landing_pad"] if isinstance(x, list) and len(x) > 0 and x[0].get("landing_pad") else None
)
missing_count = df["landingPad"].isnull().sum()
print("Missing landingPad values:", missing_count)




Total launches: 205
Falcon 1 launches: 5
Falcon 9 launches: 195
Falcon Heavy launches: 5
Missing landingPad values: 205


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


<!--## Change Log
-->


<!--

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|
-->


Copyright ©IBM Corporation. All rights reserved.
